# Pybullet GUI

In [1]:
import pybullet as pb
import numpy as np
import ArmInfo
import time
from math import *

np.set_printoptions(precision= 2)

physicsClient = pb.connect(pb.GUI,  options='--background_color_red=0.0 --background_color_green=0.66 --background_color_blue=0.66')
# joe = pb.loadURDF("Joe_URDF/robot_2arm/urdf/robot244.urdf", useFixedBase= 1)
joe = pb.loadURDF("Joe/urdf/Joe.urdf", useFixedBase= 1)
pb.setGravity(0, 0, 0)
pb.setRealTimeSimulation(1)

# Setup of Arms

In [2]:
left_arm_DH_table = np.array([[pi/2,     pi, 175,    0],
                              [pi/2,   pi/2,   0,    0],
                              [-pi/2, -pi/2,   0,  122],
                              [pi/2,   pi/2,   0,    0],
                              [pi/2,  -pi/2,   0, -257],
                              [0,     -pi/2,   0,    0],
                              [0,      pi/2,   0,  258],
                              [pi/2,   pi/2, 200,    0]])

right_arm_DH_table = np.array([[-pi/2,   -pi, 175,    0],
                               [-pi/2, -pi/2,   0,    0],
                               [-pi/2, -pi/2,   0,  122],
                               [-pi/2,  pi/2,   0,    0],
                               [pi/2,   pi/2,   0, -257],
                               [0,      pi/2,   0,    0],
                               [0,     -pi/2,   0,  258],
                               [-pi/2, -pi/2, 200,    0]])

left_arm = ArmInfo.ArmInfo(left_arm_DH_table)
right_arm = ArmInfo.ArmInfo(right_arm_DH_table)

In [3]:
def initialize():
    left_arm.delta_angle = np.array([0, 0, 0, 1e-6, -pi/2, 1e-6, 0])
    left_arm.update()
    right_arm.delta_angle = np.array([0, 0, 0, 1e-6, pi/2, 1e-6, 0])
    right_arm.update()

    print("Current Position = {}, {}".format(np.around(left_arm.current_position, 0), np.around(right_arm.current_position, 0)))
    print("Current Orientation = {}, {}".format(np.around(left_arm.get_current_orientation(), 0), np.around(right_arm.current_orientation, 0)))

    for i in range(7):
        pb.setJointMotorControl2(joe, jointIndex=i,
                                      controlMode=pb.POSITION_CONTROL,
                                      targetPosition = left_arm.delta_angle[i],
                                      force = 500)

    for i in range(7, 14):
        pb.setJointMotorControl2(joe, jointIndex=i,
                                      controlMode=pb.POSITION_CONTROL,
                                      targetPosition = right_arm.delta_angle[i-7],
                                      force = 500)

In [4]:
initialize()

Current Position = [ 458.  297. -257.], [ 458. -297. -257.]
Current Orientation = [0. 0. 0.], [ 0. -0.  0.]


In [5]:
def trajectory_planning_left(j0, ox, oy, oz, px, py, pz):
    velocity_factor = 1.0
    angular_threshold = pi/180
    linear_threshold = 1.0
    zeroth_joint_threshold = pi/180

    angular_error = sqrt(pow(ox * pi / 180 - left_arm.current_orientation[0], 2) +
                         pow(oy * pi / 180 - left_arm.current_orientation[1], 2) +
                         pow(oz * pi / 180 - left_arm.current_orientation[2], 2)) / 3
    
    linear_error = sqrt(pow(px - left_arm.current_position[0], 2) +
                        pow(py - left_arm.current_position[1], 2) +
                        pow(pz - left_arm.current_position[2], 2)) / 3

    zeroth_joint_error = (j0 * pi / 180 - pb.getJointState(joe, 0)[0])

    while angular_error > angular_threshold or linear_error > linear_threshold or abs(zeroth_joint_error) > zeroth_joint_threshold:
        # update arm state
        left_arm.delta_angle = np.array(pb.getJointStates(joe, [0, 1, 2, 3, 4, 5, 6]))[0:7, 0]
        left_arm.update()

        # update error
        angular_error = sqrt(pow(ox * pi / 180 - left_arm.current_orientation[0], 2) +
                             pow(oy * pi / 180 - left_arm.current_orientation[1], 2) +
                             pow(oz * pi / 180 - left_arm.current_orientation[2], 2)) / 3
    
        linear_error = sqrt(pow(px - left_arm.current_position[0], 2) +
                            pow(py - left_arm.current_position[1], 2) +
                            pow(pz - left_arm.current_position[2], 2)) / 3

        zeroth_joint_error = (j0 * pi / 180 - pb.getJointState(joe, 0)[0])

        # calculate linear velocity
        vel_ox = (ox * pi / 180 - left_arm.current_orientation[0]) * velocity_factor
        vel_oy = (oy * pi / 180 - left_arm.current_orientation[1]) * velocity_factor
        vel_oz = (oz * pi / 180 - left_arm.current_orientation[2]) * velocity_factor

        vel_px = (px - left_arm.current_position[0]) * velocity_factor
        vel_py = (py - left_arm.current_position[1]) * velocity_factor
        vel_pz = (pz - left_arm.current_position[2]) * velocity_factor

        vel_zeroth_joint = (j0 * pi / 180 - pb.getJointState(joe, 0)[0])

        linear_vel = np.array((vel_ox, vel_oy, vel_oz, vel_px, vel_py, vel_pz))
        
        angular_vel = np.matmul(left_arm.inverse_jacobian, linear_vel)

        # check velocity limit
        max_index = np.argmax(angular_vel)
        max_value = np.max(angular_vel)

        if abs(max_value) > pi:
            print("\tDangerous ! Stop moving.")
            break
        elif abs(max_value) > pi/6:
            angular_vel *= (pi / 6 / abs(max_value))
        
        # render in simulation
        for i in range(7):
            if i == 0:
                pb.setJointMotorControl2(joe, jointIndex=i,
                                            controlMode = pb.VELOCITY_CONTROL,
                                            targetVelocity = vel_zeroth_joint,
                                            force = 50000)

            else:
                pb.setJointMotorControl2(joe, jointIndex=i,
                                            controlMode = pb.VELOCITY_CONTROL,
                                            targetVelocity = angular_vel[i-1],
                                            force = 50000)

        time.sleep(0.1)

    for i in range(7):
        pb.setJointMotorControl2(joe, jointIndex=i,
                                        controlMode = pb.VELOCITY_CONTROL,
                                        targetVelocity = 0,
                                        force = 500)

In [6]:
def trajectory_planning_right(j0, ox, oy, oz, px, py, pz):
    velocity_factor = 1.0
    angular_threshold = pi/180
    linear_threshold = 1.0
    zeroth_joint_threshold = pi/180

    angular_error = sqrt(pow(ox * pi / 180 - right_arm.current_orientation[0], 2) +
                         pow(oy * pi / 180 - right_arm.current_orientation[1], 2) +
                         pow(oz * pi / 180 - right_arm.current_orientation[2], 2)) / 3
    
    linear_error = sqrt(pow(px - right_arm.current_position[0], 2) +
                        pow(py - right_arm.current_position[1], 2) +
                        pow(pz - right_arm.current_position[2], 2)) / 3

    zeroth_joint_error = (j0 * pi / 180 - pb.getJointState(joe, 7)[0])

    while angular_error > angular_threshold or linear_error > linear_threshold or abs(zeroth_joint_error) > zeroth_joint_threshold:
        # update arm state
        right_arm.delta_angle = np.array(pb.getJointStates(joe, [7, 8, 9, 10, 11, 12, 13]))[0:7, 0]
        right_arm.update()

        # update error
        angular_error = sqrt(pow(ox * pi / 180 - right_arm.current_orientation[0], 2) +
                             pow(oy * pi / 180 - right_arm.current_orientation[1], 2) +
                             pow(oz * pi / 180 - right_arm.current_orientation[2], 2)) / 3
    
        linear_error = sqrt(pow(px - right_arm.current_position[0], 2) +
                            pow(py - right_arm.current_position[1], 2) +
                            pow(pz - right_arm.current_position[2], 2)) / 3

        zeroth_joint_error = (j0 * pi / 180 - pb.getJointState(joe, 7)[0])

        # calculate linear velocity
        vel_ox = (ox * pi / 180 - right_arm.current_orientation[0]) * velocity_factor
        vel_oy = (oy * pi / 180 - right_arm.current_orientation[1]) * velocity_factor
        vel_oz = (oz * pi / 180 - right_arm.current_orientation[2]) * velocity_factor

        vel_px = (px - right_arm.current_position[0]) * velocity_factor
        vel_py = (py - right_arm.current_position[1]) * velocity_factor
        vel_pz = (pz - right_arm.current_position[2]) * velocity_factor

        vel_zeroth_joint = (j0 * pi / 180 - pb.getJointState(joe, 7)[0])

        linear_vel = np.array((vel_ox, vel_oy, vel_oz, vel_px, vel_py, vel_pz))
        
        angular_vel = np.matmul(right_arm.inverse_jacobian, linear_vel)

        # check velocity limit
        max_index = np.argmax(angular_vel)
        max_value = np.max(angular_vel)

        if abs(max_value) > pi:
            print("\tDangerous ! Stop moving.")
            break
        elif abs(max_value) > pi/6:
            angular_vel *= (pi / 6 / abs(max_value))
        
        # render in simulation
        for i in range(7, 14):
            if i == 7:
                pb.setJointMotorControl2(joe, jointIndex=i,
                                            controlMode = pb.VELOCITY_CONTROL,
                                            targetVelocity = vel_zeroth_joint,
                                            force = 50000)

            else:
                pb.setJointMotorControl2(joe, jointIndex=i,
                                            controlMode = pb.VELOCITY_CONTROL,
                                            targetVelocity = angular_vel[i-8],
                                            force = 50000)

        time.sleep(0.1)

    for i in range(7, 14):
        pb.setJointMotorControl2(joe, jointIndex=i,
                                        controlMode = pb.VELOCITY_CONTROL,
                                        targetVelocity = 0,
                                        force = 500)

In [7]:
initialize()

Current Position = [ 458.  297. -257.], [ 458. -297. -257.]
Current Orientation = [0. 0. 0.], [ 0. -0.  0.]


In [11]:
trajectory_planning_left(0, 0, 0, 0, 450, 300, 0)

ipykernel_launcher:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
trajectory_planning_right(0, 0, 0, 0, 350, -300, -300)

ipykernel_launcher:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [15]:
pb.setJointMotorControl2(joe, jointIndex=6,
                                        controlMode = pb.VELOCITY_CONTROL,
                                        targetVelocity = 0,
                                        force = 500)